# Proof of concept of working of AutoEncoder

In [2]:
import torch


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


class AutoEncoder(torch.nn.Module):

    def __init__(self, D_in, H, shared_decoder):
        # Encoder
        super(AutoEncoder, self).__init__()
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        # Decoder
        self.decoder = shared_decoder
        
    def forward(self, x):
        # Use encoder for encoding
        x = self.w1(x)
        x = self.activation(x)
        x = self.w2(x)
        
        # Decoder
        x = self.decoder(x)
        
        return x

In [3]:
# Dummy test for AutoEncoder
from model import gan

# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de', 'nl'], 'trgt': ['en']}


# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Create toy train dataset
rows, cols = 10, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = ae(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')
# Train
for i in range(1000):
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_ae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
    
# Report outcome after training
reproduction = ae(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

NameError: name 'AutoEncoder' is not defined

# Proof of concept of working of NoisyAutoEncoder

# Proof of concept of working of non-linear-GAN

In [4]:
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1

# Architecture
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in languages['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                               lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                               weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

# Loss function
criterion = torch.nn.BCELoss()

# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = net.generator(train_data, 'de')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(10000):

    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()

    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy())
    
# Report outcome after training
reproduction = net.generator(train_data, 'de')
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Input data:
 tensor([[0.3262, 0.3874, 0.9251, 0.9934, 0.6071, 0.8287, 0.4139, 0.2610],
        [0.2380, 0.4215, 0.3736, 0.2866, 0.7767, 0.9396, 0.7336, 0.3588],
        [0.2373, 0.5886, 0.3977, 0.5954, 0.6882, 0.9674, 0.7818, 0.3925],
        [0.8115, 0.5452, 0.9320, 0.2296, 0.3907, 0.8335, 0.2031, 0.6049],
        [0.3788, 0.5697, 0.4299, 0.4714, 0.9619, 0.8276, 0.6430, 0.5305],
        [0.4828, 0.1298, 0.5827, 0.3718, 0.0118, 0.3598, 0.8698, 0.3966],
        [0.7637, 0.3759, 0.6307, 0.4375, 0.8698, 0.5631, 0.7726, 0.0767],
        [0.6337, 0.7312, 0.2318, 0.1262, 0.5005, 0.8179, 0.9828, 0.3621],
        [0.5863, 0.8876, 0.6611, 0.0289, 0.0325, 0.6636, 0.4850, 0.3284],
        [0.2700, 0.8029, 0.4334, 0.5841, 0.5684, 0.8943, 0.2297, 0.3848]])
Reproduced data:
 tensor([[-0.0495, -0.0459,  0.0039, -0.0099, -0.0937, -0.0235, -0.0085,  0.0926],
        [-0.0465, -0.0260, -0.0129, -0.0011, -0.0874, -0.0058,  0.0174,  0.0681],
        [-0.0590, -0.0258, -0.0221, -0.0060, -0.0849, -0.0078,  

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target

Epoch 30 Generator loss: 1.0406917 Discriminator loss: 0.7972643
Epoch 31 Generator loss: 0.994096 Discriminator loss: 0.7511407
Epoch 32 Generator loss: 0.95310307 Discriminator loss: 0.7108614
Epoch 33 Generator loss: 0.93349874 Discriminator loss: 0.6732998
Epoch 34 Generator loss: 0.92253506 Discriminator loss: 0.64258176
Epoch 35 Generator loss: 0.91726446 Discriminator loss: 0.6201581
Epoch 36 Generator loss: 0.9314596 Discriminator loss: 0.6007957
Epoch 37 Generator loss: 0.9588574 Discriminator loss: 0.58492255
Epoch 38 Generator loss: 0.9764387 Discriminator loss: 0.5787733
Epoch 39 Generator loss: 0.9630799 Discriminator loss: 0.5866941
Epoch 40 Generator loss: 0.89899236 Discriminator loss: 0.6192489
Epoch 41 Generator loss: 0.8167303 Discriminator loss: 0.66918784
Epoch 42 Generator loss: 0.73013604 Discriminator loss: 0.73388565
Epoch 43 Generator loss: 0.6407397 Discriminator loss: 0.8148195
Epoch 44 Generator loss: 0.6030246 Discriminator loss: 0.8758815
Epoch 45 Generat

Epoch 156 Generator loss: 0.64293253 Discriminator loss: 0.7556182
Epoch 157 Generator loss: 0.6449634 Discriminator loss: 0.76130074
Epoch 158 Generator loss: 0.65261173 Discriminator loss: 0.7649275
Epoch 159 Generator loss: 0.66673684 Discriminator loss: 0.76564825
Epoch 160 Generator loss: 0.693386 Discriminator loss: 0.76104677
Epoch 161 Generator loss: 0.7307733 Discriminator loss: 0.75128365
Epoch 162 Generator loss: 0.77077615 Discriminator loss: 0.74002063
Epoch 163 Generator loss: 0.80915964 Discriminator loss: 0.7292516
Epoch 164 Generator loss: 0.8422743 Discriminator loss: 0.719576
Epoch 165 Generator loss: 0.86581886 Discriminator loss: 0.7120177
Epoch 166 Generator loss: 0.87612647 Discriminator loss: 0.70746815
Epoch 167 Generator loss: 0.87159747 Discriminator loss: 0.70655614
Epoch 168 Generator loss: 0.85002536 Discriminator loss: 0.7101574
Epoch 169 Generator loss: 0.8176646 Discriminator loss: 0.7168616
Epoch 170 Generator loss: 0.77722037 Discriminator loss: 0.725

Epoch 308 Generator loss: 0.45864677 Discriminator loss: 0.769395
Epoch 309 Generator loss: 0.41574144 Discriminator loss: 0.8111027
Epoch 310 Generator loss: 0.38259757 Discriminator loss: 0.8523485
Epoch 311 Generator loss: 0.3625489 Discriminator loss: 0.887613
Epoch 312 Generator loss: 0.35961133 Discriminator loss: 0.9088313
Epoch 313 Generator loss: 0.3777057 Discriminator loss: 0.9117306
Epoch 314 Generator loss: 0.41816315 Discriminator loss: 0.89758074
Epoch 315 Generator loss: 0.48293638 Discriminator loss: 0.86885816
Epoch 316 Generator loss: 0.5747997 Discriminator loss: 0.8301343
Epoch 317 Generator loss: 0.692929 Discriminator loss: 0.7887856
Epoch 318 Generator loss: 0.8305257 Discriminator loss: 0.75114334
Epoch 319 Generator loss: 0.9723752 Discriminator loss: 0.72237885
Epoch 320 Generator loss: 1.1019495 Discriminator loss: 0.70448905
Epoch 321 Generator loss: 1.1954831 Discriminator loss: 0.69754136
Epoch 322 Generator loss: 1.2363262 Discriminator loss: 0.70001924


...

Epoch 9456 Generator loss: 0.6768762 Discriminator loss: 0.6778804
Epoch 9457 Generator loss: 0.678047 Discriminator loss: 0.6821221
Epoch 9458 Generator loss: 0.66980726 Discriminator loss: 0.6945256
Epoch 9459 Generator loss: 0.67230666 Discriminator loss: 0.70434433
Epoch 9460 Generator loss: 0.67447233 Discriminator loss: 0.71589947
Epoch 9461 Generator loss: 0.63271123 Discriminator loss: 0.74675196
Epoch 9462 Generator loss: 0.68934065 Discriminator loss: 0.72486955
Epoch 9463 Generator loss: 0.7330898 Discriminator loss: 0.7189442
Epoch 9464 Generator loss: 0.7816485 Discriminator loss: 0.6945194
Epoch 9465 Generator loss: 0.8289444 Discriminator loss: 0.68048614
Epoch 9466 Generator loss: 0.84882724 Discriminator loss: 0.66791236
Epoch 9467 Generator loss: 0.847658 Discriminator loss: 0.66604424
Epoch 9468 Generator loss: 0.8249302 Discriminator loss: 0.6731056
Epoch 9469 Generator loss: 0.82720196 Discriminator loss: 0.66732776
Epoch 9470 Generator loss: 0.8033867 Discriminato

Epoch 9592 Generator loss: 0.8502469 Discriminator loss: 0.7099492
Epoch 9593 Generator loss: 0.84744084 Discriminator loss: 0.71897674
Epoch 9594 Generator loss: 0.8732556 Discriminator loss: 0.7115881
Epoch 9595 Generator loss: 0.8619524 Discriminator loss: 0.7066985
Epoch 9596 Generator loss: 0.8356608 Discriminator loss: 0.7106668
Epoch 9597 Generator loss: 0.8882438 Discriminator loss: 0.68667924
Epoch 9598 Generator loss: 0.883037 Discriminator loss: 0.67885524
Epoch 9599 Generator loss: 0.7928682 Discriminator loss: 0.6993031
Epoch 9600 Generator loss: 0.8826748 Discriminator loss: 0.65018934
Epoch 9601 Generator loss: 0.87414086 Discriminator loss: 0.64006025
Epoch 9602 Generator loss: 0.7810816 Discriminator loss: 0.6634618
Epoch 9603 Generator loss: 0.7695555 Discriminator loss: 0.6595925
Epoch 9604 Generator loss: 0.7916068 Discriminator loss: 0.6409988
Epoch 9605 Generator loss: 0.74585813 Discriminator loss: 0.6531631
Epoch 9606 Generator loss: 0.70471954 Discriminator los

Epoch 9722 Generator loss: 0.7802344 Discriminator loss: 0.6231138
Epoch 9723 Generator loss: 0.8305315 Discriminator loss: 0.60062706
Epoch 9724 Generator loss: 0.79464775 Discriminator loss: 0.6130341
Epoch 9725 Generator loss: 0.72911686 Discriminator loss: 0.6383932
Epoch 9726 Generator loss: 0.8146416 Discriminator loss: 0.5946628
Epoch 9727 Generator loss: 0.82138026 Discriminator loss: 0.5874782
Epoch 9728 Generator loss: 0.78705305 Discriminator loss: 0.59891546
Epoch 9729 Generator loss: 0.6915435 Discriminator loss: 0.64371234
Epoch 9730 Generator loss: 0.593461 Discriminator loss: 0.7113004
Epoch 9731 Generator loss: 0.40996504 Discriminator loss: 0.88615453
Epoch 9732 Generator loss: 0.3690324 Discriminator loss: 0.97326374
Epoch 9733 Generator loss: 0.4360413 Discriminator loss: 0.95204735
Epoch 9734 Generator loss: 0.6265382 Discriminator loss: 0.86348915
Epoch 9735 Generator loss: 1.0289689 Discriminator loss: 0.712396
Epoch 9736 Generator loss: 1.5240487 Discriminator l

Epoch 9845 Generator loss: 0.70049614 Discriminator loss: 0.6234869
Epoch 9846 Generator loss: 0.6632957 Discriminator loss: 0.64202374
Epoch 9847 Generator loss: 0.60083973 Discriminator loss: 0.6800369
Epoch 9848 Generator loss: 0.6209415 Discriminator loss: 0.6739998
Epoch 9849 Generator loss: 0.6284403 Discriminator loss: 0.6778884
Epoch 9850 Generator loss: 0.57414 Discriminator loss: 0.7178779
Epoch 9851 Generator loss: 0.58528525 Discriminator loss: 0.71397126
Epoch 9852 Generator loss: 0.6183527 Discriminator loss: 0.69952375
Epoch 9853 Generator loss: 0.58842665 Discriminator loss: 0.73344195
Epoch 9854 Generator loss: 0.6286393 Discriminator loss: 0.7226342
Epoch 9855 Generator loss: 0.645082 Discriminator loss: 0.72512245
Epoch 9856 Generator loss: 0.62840885 Discriminator loss: 0.74230164
Epoch 9857 Generator loss: 0.74641716 Discriminator loss: 0.68289
Epoch 9858 Generator loss: 0.68987805 Discriminator loss: 0.72256136
Epoch 9859 Generator loss: 0.7725724 Discriminator lo

Epoch 9969 Generator loss: 0.7691988 Discriminator loss: 0.62063843
Epoch 9970 Generator loss: 0.7725897 Discriminator loss: 0.6179931
Epoch 9971 Generator loss: 0.77691305 Discriminator loss: 0.6157187
Epoch 9972 Generator loss: 0.78298324 Discriminator loss: 0.6135236
Epoch 9973 Generator loss: 0.78958315 Discriminator loss: 0.6109382
Epoch 9974 Generator loss: 0.7947884 Discriminator loss: 0.6082574
Epoch 9975 Generator loss: 0.79956186 Discriminator loss: 0.6054525
Epoch 9976 Generator loss: 0.80287683 Discriminator loss: 0.6021556
Epoch 9977 Generator loss: 0.8056346 Discriminator loss: 0.59913206
Epoch 9978 Generator loss: 0.8087987 Discriminator loss: 0.5957719
Epoch 9979 Generator loss: 0.8102366 Discriminator loss: 0.59161717
Epoch 9980 Generator loss: 0.81359375 Discriminator loss: 0.5887726
Epoch 9981 Generator loss: 0.8186854 Discriminator loss: 0.58540946
Epoch 9982 Generator loss: 0.82484865 Discriminator loss: 0.5816591
Epoch 9983 Generator loss: 0.8310461 Discriminator 